In [1]:
import sympy as sp

# Exercici 1

In [37]:
def is_toeplitz( mat ):
    
    import sympy as sp
    
    result = True # benefit of the doubt
    
    for idx_1 in range( mat.shape[ 0 ] - 1 ) :
        for idx_2 in range( mat.shape[ 1 ] - 1 ) :
            diff = mat[ idx_1, idx_2 ] - mat[ idx_1 + 1, idx_2 +1 ]
            if not( sp.simplify( diff ) == 0 ) :
                result = False # violation of the condition
                break
                
    return result

# --------------------------------------------------------------------------------

def is_hermitian( mat ):
    
    import sympy as sp
    
    result = True # benefit of the doubt
    
    for idx_1 in range( mat.shape[ 0 ] ) :
        for idx_2 in ( range( mat.shape[ 1 ] ) ) :
            if (idx_2 == idx_1): # no comprovar diagonal, (si al ex.2 entrem un complex al primer element no funcionarà mai)
                continue
            diff = mat[ idx_1, idx_2 ] - sp.conjugate( mat[ idx_2, idx_1 ] )
            if not( sp.simplify( diff ) == 0 ) :
                print(idx_1, idx_2)
                result = False # violation of the condition
                break

    return result

In [38]:
from collections.abc import Iterable

def unwrap(a: Iterable): # no la utilitzo
    """
    Desfà una llista de llistes de llistes... en una sola llista
    """
    new = []
    
    for i in a:
        new = new + (unwrap(i) if isinstance(i, Iterable) else [i])
    
    return new

def toeplitz_hermitians(vec, conversor) -> list: 
    """
    Crea una matriu toeplitz hemitania (però on tu li dius la forma d'implementar el conjugat)
    """
    new = [list(vec)] # fico el primer vector talqual
    reversed_vec = list(map(conversor, reversed(vec[1:]))) # creo un vector sense el primer element pero invertit i amb el conjugat
    for i in range(1, len(vec)):
        new.append([item for item in reversed_vec[-i:]] + vec[:-i]) # vaig ficantlos de forma progressiva (cada iteracio fico un mes d'un costat i un menys de l'altre)

    return sp.Matrix(new)

def compute_herm_toeplitz_matrix(vec):
    """Crea una matriu de toeplitz hermitania on la primera fila és el paràmtetre"""
    
    new = toeplitz_hermitians(vec, sp.conjugate) 

    assert is_toeplitz(new), "Not toeplitz"
    assert is_hermitian(new), "Not hermitian"

    return new


In [39]:
def unit_test_toeplitz():
    
    import sympy as sp
    sp.init_printing()

    alpha = sp.symbols( 'alpha' )
    x = sp.symbols( 'x' )
    z = sp.symbols( 'z' )

    v = sp.Matrix( [ x + sp.conjugate( x ), sp.exp( alpha ), sp.sin( z ) ])

    mat = compute_herm_toeplitz_matrix( v )
    print(mat)
    test_1 = is_toeplitz( mat )
    test_2 = is_hermitian( mat )
    result = test_1 and test_2
    
    return result

In [40]:
unit_test_toeplitz()

Matrix([[x + conjugate(x), exp(alpha), sin(z)], [exp(conjugate(alpha)), x + conjugate(x), exp(alpha)], [sin(conjugate(z)), exp(conjugate(alpha)), x + conjugate(x)]])


True

# Exercici 2

In [42]:
def eigen_herm_toeplitz_from_complex(v):
    """
    Funcio que donat una llista de numeros complexos et construeix una matriu 
    Toeplitz Hermitania i troba els eigenvalues i els eigenvectors
    """
    mat = compute_herm_toeplitz_matrix(v) # obtinc la matriu
#     print(mat)
    eigen = mat.eigenvects() # trec els vectors propis i els valors propis en format [[(valor, vegades, vector)], ...]

    values = [val for (val, times, *_) in eigen for _ in range(times)] # obtinc els valors
    vectors = [vecs for (*_, vecs) in eigen] # obtinc els vectors
    
    mat_eigenvalues = sp.diag(*values) # fico en una matriu diagonal els valors
    
    len_values = len(values)
    
    return sp.Matrix(mat_eigenvalues), sp.Matrix(vectors).reshape(len_values, len_values) # retorno les dues matrius amb la segona amb les mesures adequades



a, b = eigen_herm_toeplitz_from_complex([1+1j, 3-4j, 6+1j])    
a, b

Matrix([[1.0 + 1.0*I, 3.0 - 4.0*I, 6.0 + 1.0*I], [3.0 + 4.0*I, 1.0 + 1.0*I, 3.0 - 4.0*I], [6.0 - 1.0*I, 3.0 + 4.0*I, 1.0 + 1.0*I]])


⎛⎡-9.00937728250052 + 1.0⋅ⅈ             0                         0           
⎜⎢                                                                            
⎜⎢            0              2.56095885910679 + 1.0⋅ⅈ             0           
⎜⎢                                                                            
⎝⎣            0                         0              9.44841842339373 + 1.0⋅

 ⎤  ⎡-0.124415001025874 + 0.580158672880267⋅ⅈ   0.543793313097663 - 0.01274661
 ⎥  ⎢                                                                         
 ⎥, ⎢-0.525885637269561 + 0.0811373291289101⋅ⅈ   0.360803514310901 - 0.5509472
 ⎥  ⎢                                                                         
ⅈ⎦  ⎣ -0.451142556369379 + 0.4015821783514⋅ⅈ    -0.441578341910388 + 0.2746140

65450152⋅ⅈ  -0.151461477513548 - 0.573692076839025⋅ⅈ⎤⎞
                                                    ⎥⎟
6129797⋅ⅈ   0.135828751923323 + 0.514479853653705⋅ⅈ ⎥⎟
                                           

In [5]:
list([1, 2, 3, 4])

[1, 2, 3, 4]

# Exercici 3

In [45]:
def compute_revo_volume(f, respect, a, b, delta): # he inclos la variable respect per si la integral es respecte altres variables
    """
    Calcula el volum del cos de revolució que representa la funció f 
    respecte la variable que li diguis i entre els valors de x a i b
    """
    
    assert abs(a - b) > delta
    
    res = sp.pi * sp.integrate(f**2, (respect, a, b)) # sp.integrate per integrar
    
    assertion = False
    
    if (a < b):
        assert res >= 0 
    else:
        assert res < 0
    
    return res

x = sp.symbols("x")

compute_revo_volume(sp.sin(x), x, 0, sp.pi, 1e-6) 